<a href="https://colab.research.google.com/github/bahramzada/azerbaijani-news-sentiment/blob/main/News_Sentiment_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd # pandas library for data manipulation (Data manipulyasiyası üçün pandas kitabxanası)
from sklearn.model_selection import train_test_split # splitting dataset into train and test sets (Veri bazasını train və test hissələrinə bölmək üçün)
from datasets import Dataset # HuggingFace Datasets for dataset handling (Datasetlərlə işləmək üçün HuggingFace Datasets)
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig # transformers library for NLP models (NLP modelləri üçün transformers kitabxanası)
import random # for generating random numbers (Təsadüfi ədədlər yaratmaq üçün)
import torch # PyTorch library for deep learning (Dərin öyrənmə üçün PyTorch kitabxanası)
import sys # system-specific parameters and functions (Sistemə aid parametrlər və funksiyalar üçün)
import re # regular expressions for text processing (Mətn emalı üçün regular ifadələr)
import unicodedata # unicode character handling (Unicode simvolları ilə işləmək üçün)
import numpy as np # numerical computations (Sayısal hesablama üçün numpy)
from sklearn.metrics import accuracy_score, f1_score # model evaluation metrics (Modelin qiymətləndirilməsi üçün ölçülər)

[My Kaggle Dataset](https://www.kaggle.com/datasets/raullte/azerbaijani-news-sentiment-dataset)

In [ ]:
df = pd.read_csv('/content/labeled-sentiment.csv') # read the CSV file into a DataFrame (CSV faylını DataFrame-ə oxuyur)
print(df.head()) # display the first 5 rows of the DataFrame (DataFrame-in ilk 5 sətrini göstərir)

  sentiment                                               text
0   Neutral  Prezident öz siyasətinin təməl prinsiplərini a...
1  Positive  Hamilə qadınlar bu xidmətlərdən pulsuz yararla...
2   Neutral               Prezident Xankəndidə qəbul keçirdi  
3   Neutral  Doktorantura üzrə bəzi imtahanların nəticələri...
4   Neutral   Bakıda bu 2 marşrutun hərəkət sxemi dəyişdirilir


In [ ]:
df['sentiment'].value_counts() # count the occurrences of each sentiment class (Hər bir sentiment sinfinin neçə dəfə təkrarlanmasını sayır)

,count
sentiment,
Positive,1029
Negative,999
Neutral,930


In [ ]:
print(f"Number of duplicate rows in 'text' column: {df['text'].duplicated().sum()}") # print the number of duplicate rows in the 'text' column ('text' sütununda neçə dublikat sətrin olduğunu çap edir)

Number of duplicate rows in 'text' column: 3


In [ ]:
duplicate_rows = df[df.duplicated(subset=['text'], keep=False)] # find all duplicate rows based on the 'text' column ('text' sütunu üzrə bütün dublikat sətrləri tapır)
display(duplicate_rows) # display the duplicate rows (Dublikat sətrləri göstərir)

,sentiment,text
208,Negative,İstirahətə gedən qadın eyvandan yıxılıb öldü
329,Positive,Yeni idman kompleksi açıldı
397,Neutral,Hava limanında yeni terminal istifadəyə verildi
2872,Neutral,Hava limanında yeni terminal istifadəyə verildi
2879,Negative,İstirahətə gedən qadın eyvandan yıxılıb öldü
2904,Positive,Yeni idman kompleksi açıldı


In [ ]:
df = df.drop_duplicates().reset_index(drop=True) # remove duplicate rows and reset index (Dublikat sətrləri silir və indeksləri yeniləyir)
print(f"Number of duplicate rows after dropping: {df.duplicated().sum()}") # print the number of duplicates after removal (Dublikatlar silindikdən sonra neçə dublikat qaldığını çap edir)

Number of duplicate rows after dropping: 0


In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True) # shuffle the DataFrame rows randomly and reset index (DataFrame sətrlərini təsadüfi qarışdırır və indeksləri yeniləyir)
display(df.head()) # display the first 5 rows of the shuffled DataFrame (Qarışdırılmış DataFrame-in ilk 5 sətrini göstərir)

,sentiment,text
0,Negative,Yeni qanunvericilik biznes mühitini pisləşdirdi
1,Negative,Elektrik enerjisi təchizatında tez-tez kəsilmə...
2,Negative,İsmayıllıda hotel yandı
3,Positive,Universitet tələbələri üçün təqaüd artırıldı
4,Negative,Rayonda bir ailə yanğında evini itirib


In [ ]:
# Azerbaijani stopwords set (Azərbaycan dilində stopwordslər siyahısı)
az_stopwords = set([
    "və", "bu", "bir", "ilə", "üçün", "ki", "həm", "daha", "da", "ən", "də", "onun", "kimi", "olan", "hansı", "hər"
])

def preprocess_text(text):
    text = unicodedata.normalize('NFKC', text) # normalize unicode characters (Unicode simvollarını normallaşdırır)
    text = text.replace('I', 'ı').replace('İ', 'i') # replace capital I and İ with lowercase equivalents (Böyük I və İ hərflərini kiçik formalarına çevirir)
    text = text.lower() # convert text to lowercase (Mətni kiçik hərflərə çevirir)

    text = re.sub(r'i̇', 'i', text) # fix dotted i probelm (nöqtəli i hərfini düzəldir)
    # Remove punctuation and numbers (Nişan və rəqəmləri silir)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', '', text)
    # Remove extra spaces (Artıq boşluqları silir)
    text = re.sub(r'\s+', ' ', text).strip()

    words = [w for w in text.split() if w not in az_stopwords] # remove stopwords from text (Stopwordsləri mətnin içindən çıxarır)
    return ' '.join(words) # join cleaned words into a single string (Təmizlənmiş sözləri bir mətndə birləşdirir)

df['clean_text'] = df['text'].apply(preprocess_text) # preprocess all texts and create new column (Bütün mətnləri təmizləyib yeni sütun yaradır)
df = df[['sentiment','clean_text']] # keep only sentiment and clean_text columns (Yalnız sentiment və təmizlənmiş mətn sütunlarını saxlayır)
df = df.rename(columns={'clean_text': 'text'}) # rename 'clean_text' to 'text' (clean_text sütununu text adlandırır)
display(df.head()) # display first 5 rows of the cleaned DataFrame (Təmizlənmiş DataFrame-in ilk 5 sətrini göstərir)

,sentiment,text
0,Negative,yeni qanunvericilik biznes mühitini pisləşdirdi
1,Negative,elektrik enerjisi təchizatında tez tez kəsilmə...
2,Negative,ismayıllıda hotel yandı
3,Positive,universitet tələbələri təqaüd artırıldı
4,Negative,rayonda ailə yanğında evini itirib


In [ ]:
# 2. Label mapping (str → int)
label2id = {label: idx for idx, label in enumerate(sorted(df['sentiment'].unique()))}
id2label = {v: k for k, v in label2id.items()}
df['label'] = df['sentiment'].map(label2id)

In [ ]:
# 3. Train/test partition (Train/test bölməsi)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# 4. Tokenizer and model
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
config.hidden_dropout_prob = 0.35  # default 0.1, artırdıq
config.attention_probs_dropout_prob = 0.35  # default 0.1, artırdıq
config.num_labels = len(label2id)
config.id2label = id2label
config.label2id = label2id


model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=config,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    # tokenize the input text with truncation and padding (Giriş mətnini truncation və padding ilə tokenizasiya edir)

train_dataset = train_dataset.map(preprocess_function, batched=True)
# apply preprocessing to the training dataset in batches (Preprocessing-i train datasetinə batch şəklində tətbiq edir)

test_dataset = test_dataset.map(preprocess_function, batched=True)
# apply preprocessing to the test dataset in batches (Preprocessing-i test datasetinə batch şəklində tətbiq edir)

Map:   0%|          | 0/2364 [00:00<?, ? examples/s]

Map:   0%|          | 0/591 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./az_sentiment_bert",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=15,  # Epoch sayını azaltdıq
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,   # Weight decay əlavə etdik
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Early stopping üçün
    save_total_limit=3,
    seed=42
)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted", zero_division=0),
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.087556,0.394247,0.277242
2,1.099200,0.828473,0.648054,0.628039
3,1.099200,0.585682,0.752961,0.750934
4,0.825900,0.534992,0.803723,0.803288
5,0.825900,0.500208,0.830795,0.830341
6,0.644200,0.570792,0.807107,0.808181
7,0.539800,0.520360,0.827411,0.825247
8,0.539800,0.540343,0.827411,0.827568
9,0.470300,0.519809,0.835871,0.835880
10,0.470300,0.560466,0.849408,0.849154


TrainOutput(global_step=4440, training_loss=0.5782580916945999, metrics={'train_runtime': 1500.5397, 'train_samples_per_second': 23.631, 'train_steps_per_second': 2.959, 'total_flos': 2332500448158720.0, 'train_loss': 0.5782580916945999, 'epoch': 15.0})

In [ ]:
trainer.save_model("./az_sentiment_bert")
tokenizer.save_pretrained("./az_sentiment_bert")

('./az_sentiment_bert/tokenizer_config.json',
 './az_sentiment_bert/special_tokens_map.json',
 './az_sentiment_bert/vocab.txt',
 './az_sentiment_bert/added_tokens.json',
 './az_sentiment_bert/tokenizer.json')

In [ ]:
# Progress bar function (Səviyyə barı funksiyası)
def level_bar(current, total, bar_length=50):
    percent = current / total
    filled_length = int(bar_length * percent)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\rProgress: |{bar}| {percent*100:.1f}% ({current}/{total})')
    sys.stdout.flush()
    if current == total:
        print()

# Load the model and tokenizer (Model və tokenizatoru yükləyir)
model_dir = "./az_sentiment_bert" # directory where the trained model is saved (Təlim edilmiş modelin saxlanıldığı qovluq)
tokenizer = AutoTokenizer.from_pretrained(model_dir) # load tokenizer from the directory (Tokenizatoru qovluqdan yükləyir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir) # load sequence classification model from the directory (Sıra təsnifat modelini qovluqdan yükləyir)

# Label mapping (Etiketlərin xəritələnməsi)
id2label = {0: "Negative", 1: "Neutral", 2: "Positive"} # map label ids to label names (Label id-ləri etiket adlarına xəritələyir)
label2id = {v: k for k, v in id2label.items()} # map label names to label ids (Etiket adlarını id-lərə xəritələyir)

#  mixed news headlines for testing and their correct sentiments (Test üçün qarışıq xəbər başlığı və onların doğru sentimentləri)
test_data = [
    ("Futbol komandamız növbəti mərhələyə keçdi", "Positive"),
    ("Sosial şəbəkələrdə yayılmış saxta xəbərlər tənqid olunur", "Negative"),
    ("Məktəblərdə yeni tədris ilinə hazırlıqlar görülür", "Neutral"),
    ("İqlim dəyişikliyi ilə bağlı yeni tədbirlər elan olundu", "Neutral"),
    ("Şəhər yollarında son günlər tıxaclar artıb", "Negative"),
    ("Gənc ixtiraçı beynəlxalq müsabiqədə qalib oldu", "Positive"),
    ("Qiymət artımı vətəndaşların narazılığına səbəb oldu", "Negative"),
    ("Universitet tələbələri üçün yeni təqaüd proqramı açıldı", "Positive"),
    ("Elektrik enerjisində fasilələr bir neçə rayonda müşahidə edildi", "Negative"),
    ("Kitab sərgisi minlərlə ziyarətçi cəlb etdi", "Positive"),
    ("Mədəniyyət festivalı rəngarəng proqramla baş tutdu", "Positive"),
    ("Dərman qıtlığı ilə bağlı əhalidə narahatlıq artır", "Negative"),
    ("İşsizlik səviyyəsi son aylarda azalıb", "Positive"),
    ("Milli parkda yeni ekoturizm marşrutu istifadəyə verildi", "Positive"),
    ("İnternet sürətinin zəifliyi istifadəçiləri narahat edir", "Negative"),
    ("Təhsil sistemində islahatlar gözlənilir", "Neutral"),
    ("Kənd təsərrüfatında məhsuldarlıq rekord səviyyəyə çatıb", "Positive"),
    ("Avtomobil satışlarında kəskin azalma qeydə alınıb", "Negative"),
    ("Uşaq bağçalarında yerlərlə bağlı problem yaşanır", "Negative"),
    ("Sənaye müəssisələri yeni texnologiyalarla təchiz olunur", "Positive"),
    ("İş adamları üçün vergi güzəştləri tətbiq edildi", "Positive"),
    ("Dənizdə balıq ehtiyatları azalır", "Negative"),
    ("İdmançımız Avropa çempionatında medal qazandı", "Positive"),
    ("Təhlükəli yol şəraiti sürücülər üçün risk yaradır", "Negative"),
    ("Bank sektorunda sabitlik müşahidə olunur", "Positive"),
    ("Məktəblərdə gigiyena qaydaları gücləndirilib", "Neutral"),
    ("Tələbələrin imtahan nəticələri açıqlandı", "Neutral"),
    ("Bəzi bölgələrdə su qıtlığı problemi var", "Negative"),
    ("Dövlət xəstəxanalarında yeni avadanlıqlar istifadəyə verildi", "Positive"),
    ("Muzeydə nadir eksponatlar sərgilənir", "Positive"),
    ("İşçilərin maaşlarının gecikməsi narazılıq yaradır", "Negative"),
    ("Qida məhsullarının keyfiyyəti ilə bağlı monitorinq aparılır", "Neutral"),
    ("Müəllimlərin işə qəbul prosesi dəyişdi", "Neutral"),
    ("Ərzaq qiymətlərində sabitlik müşahidə olunur", "Neutral"),
    ("Əhali arasında sağlam həyat tərzi təşviq olunur", "Positive"),
    ("Rəqəmsal texnologiyaların inkişafı sürətlənir", "Positive"),
    ("Dövlət büdcəsində kəsir yaranıb", "Negative"),
    ("Ekoloji problemlərlə bağlı tədbirlər görülür", "Positive"),
    ("Şəhər parklarında abadlıq işləri aparılır", "Positive"),
    ("Diaqnostika mərkəzində uzun növbələr yaranıb", "Negative"),
    ("Qaz təchizatı ilə bağlı nasazlıqlar aradan qaldırılıb", "Positive"),
    ("Kənd yerlərində internet çəkilişi genişləndirilir", "Positive"),
    ("Məktəblilərin beynəlxalq olimpiadada uğurları", "Positive"),
    ("Siyasi gərginlik regionda narahatlıq yaradıb", "Negative"),
    ("Tələbələr üçün yeni yataqxana istifadəyə verildi", "Positive"),
    ("Sərnişin avtobuslarında sıxlıq müşahidə olunur", "Negative"),
    ("Təhlükəsizlik tədbirlərinin gücləndirilməsi planlaşdırılır", "Positive"),
    ("Bəzi apteklərdə dərman qıtlığı müşahidə edilir", "Negative"),
    ("Turizm mövsümündə ziyarətçi sayı rekord həddə çatdı", "Positive"),
    ("İş yerlərində əmək şəraiti yaxşılaşdırılır", "Positive"),
    ("Ətraf mühitin qorunması üçün yeni layihələr hazırlanır", "Positive"),
    ("Ödənişsiz tibbi xidmətlər genişləndirildi", "Positive"),
    ("Gənclər arasında oxu mədəniyyəti artır", "Positive"),
    ("Səhiyyə sektorunda bəzi problemlər qalmaqdadır", "Negative"),
    ("Kənd sakinləri yeni su mənbəyindən istifadə edir", "Positive"),
    ("Enerji qiymətlərinin qalxması narazılığa səbəb olub", "Negative"),
    ("Müəllimlərin peşəkar inkişafı üçün kurslar təşkil edilir", "Positive"),
    ("Yol infrastrukturunun təkmilləşdirilməsi davam edir", "Positive"),
    ("Sosial dəstək proqramlarının əhatə dairəsi artır", "Positive"),
    ("Hava şəraitinin kəskin dəyişməsi kənd təsərrüfatına təsir edib", "Negative"),
    ("Gənclər üçün innovasiya mərkəzi açıldı", "Positive"),
    ("Əhalinin məmnuniyyəti ilə bağlı sorğu keçirildi", "Neutral"),
    ("İT sahəsində yeni iş yerləri yaradılır", "Positive"),
    ("Bəzi rayonlarda elektrik kəsintisi baş verib", "Negative"),
    ("Turistlər üçün yeni marşrutlar təklif olunur", "Positive"),
    ("Mədəni tədbirlər pandemiyaya görə məhdudlaşdırılıb", "Negative"),
    ("Sərgilərdə yerli sənətkarlar iştirak edir", "Positive"),
    ("Müəssisələrin vergi borcu artıb", "Negative"),
    ("Kənd yolları asfaltlanır", "Positive"),
    ("Qida təhlükəsizliyi ilə bağlı yeni qaydalar tətbiq edilib", "Neutral"),
    ("Əmək bazarında yeni imkanlar yaranır", "Positive"),
    ("Təqaüdçülərin sosial təminatı gücləndirilir", "Positive"),
    ("Sənaye tullantılarının azalması üçün tədbirlər görülür", "Positive"),
    ("Banklar yeni kredit məhsulları təqdim edir", "Positive"),
    ("Rəqəmsal təhsil platformaları genişlənir", "Positive"),
    ("Bəzi bölgələrdə epidemiyaya qarşı tədbirlər görülür", "Negative"),
    ("Şəhər nəqliyyatında dəyişikliklər gözlənilir", "Positive"),
    ("İdman tədbirləri geniş ictimai maraq doğurur", "Positive"),
    ("İş yerlərinin bağlanması işsizliyi artırıb", "Negative"),
    ("Muzeydə təmir işləri başa çatıb", "Neutral"),
    ("Turizm sektorunda yeni investisiyalar", "Positive"),
    ("Əhalinin sağlamlığı üçün maarifləndirici proqramlar keçirilir", "Positive"),
    ("Gənclər arasında işsizlik artıb", "Negative"),
    ("Dövlət qurumlarında elektron xidmətlərin sayı artır", "Positive"),
    ("Əmək haqlarının gecikməsi narazılıq yaradır", "Negative"),
    ("Yeni tədris proqramları təsdiqləndi", "Neutral"),
    ("Kənd təsərrüfatı məhsullarının ixracı artır", "Positive"),
    ("İctimai nəqliyyatda problemlər müşahidə olunur", "Negative"),
    ("Ərzaq məhsullarında keyfiyyət yoxlamaları aparılır", "Positive"),
    ("Şagirdlər üçün pulsuz dərsliklər paylanır", "Positive"),
    ("Pandemiya səbəbindən tədbirlər ləğv edildi", "Negative"),
    ("Məktəblərdə yenilikçi tədris metodları tətbiq olunur", "Positive"),
    ("İnşaat sektorunda canlanma baş verib", "Positive"),
    ("Bəzi rayonlarda su təchizatı dayandırılıb", "Negative"),
    ("Ətraf mühitin mühafizəsi üçün könüllülər cəlb olunur", "Positive"),
    ("Sərnişinlər üçün yeni güzəştlər tətbiq edilib", "Positive"),
    ("İdmançıların beynəlxalq yarışlardakı uğurları", "Positive"),
    ("Əhalinin aztəminatlı təbəqələri üçün sosial yardım proqramı genişləndirilib", "Positive"),
    ("Kənd təsərrüfatında məhsuldarlığın azalması narahatlıq yaradıb", "Negative"),
    ("Bank sektorunda rəqəmsal transformasiya sürətlənir", "Positive"),
    ("Azərbaycan iqtisadiyyatında böyük inkişaf qeydə alınıb", "Positive"),
    ("Şəhərdə yol qəzası nəticəsində 3 nəfər yaralanıb", "Negative"),
    ("Milli kitabxanada yeni kitab fondu təqdim edildi", "Neutral"),
    ("Olimpiya Oyunlarında qızıl medal qazandıq", "Positive"),
    ("İqlim dəyişikliyi ekoloji böhrana səbəb olur", "Negative"),
    ("Bakıda metro xəttinin genişləndirilməsi planlaşdırılır", "Positive"),
    ("Elm sahəsində yeni kəşflər edildi", "Positive"),
    ("Kiberhücumlar artmaqda davam edir", "Negative"),
    ("Turizm mövsümü rəsmi olaraq açıldı", "Neutral"),
    ("Yeni texnologiya mərkəzi istifadəyə verildi", "Positive"),
    ("Təbii fəlakət səbəbindən evlər zədələnib", "Negative"),
    ("Mədəniyyət həftəsi keçiriləcək", "Neutral"),
    ("İdman yarışlarında qələbə qazandıq", "Positive"),
    ("İnflyasiya səviyyəsi artıb", "Negative"),
    ("Yeni park kompleksi tikilir", "Neutral"),
    ("Beynəlxalq müsabiqədə birinci yer aldıq", "Positive"),
    ("Şəhərdə elektrik kəsilmələri müşahidə olunur", "Negative"),
    ("Universitetdə yeni fakültə açıldı", "Neutral"),
    ("İxracat göstəriciləri artıb", "Positive"),
    ("Mənzil qiymətləri rekord həddə çatıb", "Negative"),
    ("Yeni teatr mövsümü başlayır", "Neutral"),
    ("Elm və texnologiya sahəsində beynəlxalq əməkdaşlıq", "Positive"),
    ("Narkotik maddələrin satışı halları artıb", "Negative"),
    ("Səhiyyə sistemində yeniliklər tətbiq olunur", "Positive"),
    ("Milli komandamız dünya reytinqində yüksəldi", "Positive"),
    ("İşsizlik statistikası narahatedici göstəricilər verir", "Negative"),
    ("Yeni muzey ekspozisiyası açıldı", "Neutral"),
    ("Enerji sahəsində yeni layihələr başladı", "Positive"),
    ("Dəniz suyu çirklənməsi problemli hala çatıb", "Negative"),
    ("Şəhər mərkəzində yenidənqurma işləri aparılır", "Neutral"),
    ("Beynəlxalq mükafat qazandıq", "Positive"),
    ("Yol-nəqliyyat hadisələri artmaqdadır", "Negative"),
    ("Milli bayramın qeyd olunması planlaşdırılır", "Neutral"),
    ("İnnovasiya mərkəzi uğurla fəaliyyət göstərir", "Positive"),
    ("Məhkəmə sistemində korrupsiya halları aşkar edilib", "Negative"),
    ("Şəhər nəqliyyatında yeniliklər tətbiq edilir", "Positive"),
    ("Kənd təsərrüfatında məhsuldarlıq artıb", "Positive"),
    ("Mağaza qarətçiliyi halları çoxalıb", "Negative"),
    ("Dövlət qurumlarında rəqəmsallaşma davam edir", "Neutral"),
    ("Beynəlxalq investisiyalar artırılıb", "Positive"),
    ("Su təchizatında problemlər yaranıb", "Negative"),
    ("Yeni təhsil proqramları təqdim edildi", "Positive"),
    ("Turizm gəlirləri rekord səviyyəyə çatıb", "Positive"),
    ("Atmosfer çirklənməsi səviyyəsi yüksəlir", "Negative"),
    ("Yeni ictimai nəqliyyat xətləri açılır", "Positive"),
    ("Elm olimpiadasında şagirdlərimiz qalib gəldi", "Positive"),
    ("Tibbi mərkəzlərdə növbə problemi davam edir", "Negative"),
    ("Mədəni irsin qorunması üçün yeni proqram", "Positive"),
    ("İxrac potensialı genişləndirilir", "Positive"),
    ("Kriminal halların sayı artmaqdadır", "Negative"),
    ("Rəqəmsal hökumət xidmətləri təkmilləşdirilir", "Positive"),
    ("Yeni iş yerlərinin açılması planlaşdırılır", "Positive"),
    ("Əczaxana sektorunda dərman çatışmazlığı", "Negative"),
    ("Milli arxiv sistemində modernləşmə aparılır", "Positive"),
    ("Beynəlxalq əməkdaşlıq müqaviləsi imzalandı", "Positive"),
    ("Şəhər mərkəzində tıxac problemi davam edir", "Negative"),
    ("Yeni sənət qalereya kompleksi təqdim edildi", "Neutral"),
    ("Elmi tədqiqat institutunda böyük uğur əldə edildi", "Positive"),
    ("Bankların kredit şərtlərində sərtləşmə müşahidə olunur", "Negative"),
    ("Şəhər bağlarının yenidən qurulması başladı", "Positive"),
    ("Milli mətbəximiz beynəlxalq tanınma qazandı", "Positive"),
    ("Məktəblərdə təhlükəsizlik səviyyəsi azalıb", "Negative"),
    ("Ictimai xidmətlərin keyfiyyəti artırılır", "Positive"),
    ("Enerji sektorunda innovativ həllər tətbiq edilir", "Positive"),
    ("Pərakəndə ticarətdə satış həcmi azalıb", "Negative"),
    ("Yeni incəsənət mərkəzi inşa edilir", "Neutral"),
    ("Startap ekosistemində böyük irəliləyiş var", "Positive"),
    ("Kibertəhlükəsizlik səviyyəsi narahatedicidir", "Negative"),
    ("Şəhər infrastrukturunun modernləşdirilməsi davam edir", "Positive"),
    ("Beynəlxalq sərgidə ölkəmiz uğurla təmsil olundu", "Positive"),
    ("İctimai nəqliyyatda gecikmələr tez-tez baş verir", "Negative"),
    ("Milli dildə yeni proqram təminatı hazırlanır", "Neutral"),
    ("Turizm infrastrukturu təkmilləşdirilir", "Positive"),
    ("Qida təhlükəsizliyi sahəsində problemlər aşkar edilib", "Negative"),
    ("Gənclər üçün yeni peşə hazırlığı proqramları", "Positive"),
    ("Beynəlxalq reytinqdə mövqeyimiz yaxşılaşdı", "Positive"),
    ("Yaşlı əhali üçün sosial xidmətlərdə çatışmazlıq", "Negative"),
    ("Dövlət qurumlarında şəffaflıq artırılır", "Positive"),
    ("Elmi jurnal beynəlxalq indeksə daxil edildi", "Positive"),
    ("Şəhər mərkəzində zibil yığılması problemi", "Negative"),
    ("Yeni kitabxana filialları açılır", "Positive"),
    ("Beynəlxalq investorların marağı artıb", "Positive"),
    ("Məktəb binalarının vəziyyəti qənaətbəxş deyil", "Negative"),
    ("Tarixi abidələrin bərpası proqramı başladı", "Positive"),
    ("İnnovasiya sahəsində patent sayı artıb", "Positive"),
    ("Əhalinin alıcılıq qabiliyyəti azalıb", "Negative"),
    ("Mədəniyyət və sənət festivalı keçiriləcək", "Neutral"),
    ("Rəqəmsal texnologiyalar sahəsində irəliləyiş", "Positive"),
    ("Əmək bazarında diskriminasiya halları müşahidə olunur", "Negative"),
    ("Yeni ictimai məkanların yaradılması planlaşdırılır", "Neutral"),
    ("Beynəlxalq təhsil proqramında iştirak edirik", "Positive"),
    ("Təbii ehtiyatların istifadəsində problemlər var", "Negative"),
    ("Şəhər planlamasında yeni yanaşmalar tətbiq olunur", "Positive"),
    ("Elm və texnologiya parkında yeni şirkətlər açıldı", "Positive"),
    ("Sosial müdafiə sistemində boşluqlar aşkar edilib", "Negative"),
    ("Milli ədalət sistemində islahatlar aparılır", "Neutral"),
    ("Beynəlxalq təşkilatla əməkdaşlıq genişlənir", "Positive"),
    ("Şəhər nəqliyyatında ekoloji problemlər yaranıb", "Negative"),
    ("Yeni idman kompleksi istifadəyə verildi", "Neutral"),
    ("İqtisadi artım templəri sürətlənir", "Positive"),
    ("Məhkəmə qərarlarının icrası gecikir", "Negative"),
    ("Mədəni müəssisələrdə yenidənqurma işləri", "Positive"),
    ("Beynəlxalq ticarət əlaqələri güclənir", "Positive"),
    ("Şəhər mərkəzində park yerlərinin çatışmazlığı", "Negative"),
    ("Yeni təhsil standartları təqdim edildi", "Neutral")
]

# shuffle test_data randomly (test_data-ni təsadüfi qarışdırır)
test_data = random.sample(test_data, len(test_data))

def predict_sentiment(text):
    # tokenize the input text and prepare for model (giriş mətnini tokenizasiya edir və modelə hazırlayır)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    with torch.no_grad():
        # get predictions from the model (modeldən proqnozları alır)
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = logits.argmax(dim=-1).item()
    return id2label[predicted_class_id] # return the label name (etiket adını qaytarır)

# track correct and incorrect predictions (düzgün və səhv proqnozları izləyir)
correct = 0
wrong_predictions = []

total = len(test_data)

for idx, (text, true_label) in enumerate(test_data, 1):
    # predict sentiment for each text (hər mətn üçün sentiment proqnozlaşdırır)
    pred_label = predict_sentiment(text)
    if pred_label == true_label:
        correct += 1
    else:
        wrong_predictions.append((text, true_label, pred_label))
    level_bar(idx, total)  # show progress (prosesin gedişatını göstərir)

accuracy = correct / total * 100
print(f"\nThe model's accuracy percentage: {accuracy:.2f}%") # print accuracy (dəqiqliyi çap edir)

# show incorrect predictions at the end (sonda səhv proqnozları göstərir)
if wrong_predictions:
    print(f"\nThe model's incorrect predictions ({len(wrong_predictions)}):")
    for text, true_label, pred_label in wrong_predictions:
        # print(text)
        print(f"Text: {text}")
        print(f"True label: {true_label} | Predicted label: {pred_label}")
        print("---")
else:
    print("\nThe model correctly predicted all the headlines!") # model bütün başlıqları düzgün proqnozlaşdırdı

Progress: |██████████████████████████████████████████████████| 100.0% (205/205)

The model's accuracy percentage: 84.39%

The model's incorrect predictions (32):
Text: Yeni ictimai məkanların yaradılması planlaşdırılır
True label: Neutral | Predicted label: Positive
---
Text: Yeni iş yerlərinin açılması planlaşdırılır
True label: Positive | Predicted label: Neutral
---
Text: Hava şəraitinin kəskin dəyişməsi kənd təsərrüfatına təsir edib
True label: Negative | Predicted label: Positive
---
Text: Beynəlxalq əməkdaşlıq müqaviləsi imzalandı
True label: Positive | Predicted label: Neutral
---
Text: Atmosfer çirklənməsi səviyyəsi yüksəlir
True label: Negative | Predicted label: Positive
---
Text: Kiberhücumlar artmaqda davam edir
True label: Negative | Predicted label: Positive
---
Text: Qaz təchizatı ilə bağlı nasazlıqlar aradan qaldırılıb
True label: Positive | Predicted label: Negative
---
Text: Bank sektorunda sabitlik müşahidə olunur
True label: Positive | Predicted label: Negative
---
